# Многоклассовая классификация.

Посмотрим на примере алгоритма логистической регрессии и метода опорных векторов, как работать с различными методами многоклассовой классификации.

### 1.
Вспомните датасет Wine. Загрузите его, разделите на тренировочную и тестовую выборки (random_state=17), используя только [9, 11, 12] признаки.

In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

In [2]:
wine_dataset = load_wine()
wine_dataset.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names'])

In [3]:
x_train, x_test, y_train, y_test = train_test_split(wine_dataset.data[:, [9, 11, 12]],
                                                   wine_dataset['target'],
                                                   random_state=17)

**Задайте тип кросс-валидации с помощью StratifiedKFold: 5-кратная, random_state=17.**

In [4]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [5]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

### 2.
Обучите логистическую регрессию (LogisticRegression) с параметром C по умолчанию и random_state=17. Укажите гиперпараметр multi_class='ovr' - по умолчанию многие классификаторы используют именно его. С помощью cross_val_score сделайте кросс-валидацию (используйте объект skf) и выведите среднюю долю правильных ответов на ней (используйте функцию mean). Отдельно выведите долю правильных ответов на тестовой выборке.

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

In [7]:
lr = LogisticRegression(random_state=17, multi_class='ovr')
scores = cross_val_score(lr, x_train, y_train, cv=skf)
scores.mean()

0.9096866096866097

In [8]:
scores_test = cross_val_score(lr, x_test, y_test, cv=skf)
scores.mean()

0.9096866096866097

### 3.
Обучите метод опорных векторов (SVC) с random_state=17 и остальными параметрами по умолчанию. Этот метод при мультиклассовой классификации также использует метод "ovr". Сделайте кросс-валидацию (используйте skf) и, как и в предыдущем пункте, выведите среднюю долю правильных ответов на ней. Отдельно выведите долю правильных ответов на тестовой выборке.

In [9]:
from sklearn.svm import SVC

In [12]:
svc = SVC(random_state=17)

svc_scores = cross_val_score(svc, x_train, y_train, cv=skf)
svc_scores.mean()

0.6923076923076923

In [14]:
svc_scores_test = cross_val_score(svc, x_test, y_test, cv=skf)
svc_scores_test.mean()

0.7555555555555554

Как видно из полученной метрики, на тестовой выборке метод с гиперпараметрами по умолчанию работает явно намного хуже логистической регрессии. В целом, SVM достаточно плохо масштабируется на размер обучающего набора данных (как видно, даже с тремя признаками он работает не очень хорошо), но благодаря возможности выбора различных ядер (функций близости, которые помогают разделять данные) и другим гиперпараметрам SVM можно достаточно точно настроить под определенный вид данных. Подробнее на этом останавливаться в контексте данного урока не будем.

### 4.
Для предсказаний обеих моделей постройте матрицу ошибок (confusion matrix) и напишите, какие классы каждая из моделей путает больше всего между собой.

In [15]:
from sklearn.metrics import classification_report, confusion_matrix

In [21]:
lr_model = lr.fit(x_train, y_train)
lr_pred = lr_model.predict(x_test)
conf_mtrx = confusion_matrix(y_test, lr_pred)
conf_mtrx



array([[ 9,  0,  0],
       [ 0, 19,  0],
       [ 0,  4, 13]], dtype=int64)

Объекты 1 класса (4) чаще всего путают с объектами 2 класса (13)

In [22]:
svc_model = svc.fit(x_train, y_train)
svc_pred = svc_model.predict(x_test)
conf_mtrx_svc = confusion_matrix(y_test, svc_pred)
conf_mtrx_svc

array([[ 9,  0,  0],
       [ 0, 19,  0],
       [ 2, 15,  0]], dtype=int64)

Объекты 0 класса (2) чаще всего путают с объектами 1 класса (15)

Примечательно, что разные методы дают разные результаты классификации 2 класса

### 5.
Для каждой модели выведите classification report.

In [24]:
lr_report = classification_report(y_test, lr_pred, output_dict=True)
lr_report

{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 9},
 '1': {'precision': 0.8260869565217391,
  'recall': 1.0,
  'f1-score': 0.9047619047619047,
  'support': 19},
 '2': {'precision': 1.0,
  'recall': 0.7647058823529411,
  'f1-score': 0.8666666666666666,
  'support': 17},
 'accuracy': 0.9111111111111111,
 'macro avg': {'precision': 0.9420289855072465,
  'recall': 0.9215686274509803,
  'f1-score': 0.9238095238095237,
  'support': 45},
 'weighted avg': {'precision': 0.9265700483091788,
  'recall': 0.9111111111111111,
  'f1-score': 0.9094179894179895,
  'support': 45}}

In [25]:
svc_report = classification_report(y_test, svc_pred, output_dict=True)
svc_report

C:\Anaconda 3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda 3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda 3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'precision': 0.8181818181818182,
  'recall': 1.0,
  'f1-score': 0.9,
  'support': 9},
 '1': {'precision': 0.5588235294117647,
  'recall': 1.0,
  'f1-score': 0.7169811320754718,
  'support': 19},
 '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 17},
 'accuracy': 0.6222222222222222,
 'macro avg': {'precision': 0.4590017825311943,
  'recall': 0.6666666666666666,
  'f1-score': 0.538993710691824,
  'support': 45},
 'weighted avg': {'precision': 0.3995840760546643,
  'recall': 0.6222222222222222,
  'f1-score': 0.4827253668763103,
  'support': 45}}